In [1]:
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf


#티스토리 업로드시 최적화 코드
#from IPython.core.display import display, HTML
#display(HTML("<style>.container {width:90% !important;}</style>"))

In [1]:
import pandas as pd

In [2]:
file_dir = r'C:\Users\qusru\open'

In [3]:
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [5]:
train = pd.read_csv(file_dir + r'\train.csv')
test = pd.read_csv(file_dir + r'\test.csv')
sample_submission = pd.read_csv(file_dir+r'\sample_submission.csv', index_col = 0)

기존 train 데이터로 self training을 진행하여 null이 있는 열을 학습시켜야함.
순서는 null이 적은 값부터 많은 순서대로 학습 <br>
Airline > Carrier_ID(DOT) > Carrier_Code(IATA) > Estimated_Departure_Time > Estimated_Arrival_Time > delay <br>  

In [6]:
import datetime
def to_time(time_list):
    Time = pd.Series(time_list).astype(str).str.zfill(4)
    Time = Time.replace('2400','0000')
    return [datetime.datetime.strptime(i, '%H%M').strftime("%H:%M") if i != '0nan' else np.NaN for i in Time] 

time_list = [i if str(i) == 'nan' else str(int(i))  for i in train['Estimated_Departure_Time'] ]
time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in train['Estimated_Arrival_Time'] ]
train['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
train['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,...,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,...,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,NaN,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,...,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,07:40,10:24
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,...,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,16:10,18:05
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,...,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,09:05,17:35
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,...,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,09:00,10:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,...,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,09:36,12:43
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,...,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,09:20,10:28
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,...,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,08:00,13:40
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,...,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16:13,18:24


In [7]:
time_list = [i if str(i) == 'nan' else str(int(i))  for i in test['Estimated_Departure_Time'] ]
time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in test['Estimated_Arrival_Time'] ]

test['Estimated_Departure_Time_HH:MM'] = to_time(time_list)
test['Estimated_Arrival_Time_HH:MM'] = to_time(time_list1)

In [8]:
train['Estimated_Departure_HH'] = [int(i.split(':')[0]) if str(i) != 'nan' else np.NaN for i in train['Estimated_Departure_Time_HH:MM']]
train['Estimated_Departure_MM'] = [int(i.split(':')[1]) if str(i) != 'nan' else np.NaN for i in train['Estimated_Departure_Time_HH:MM']]

train['Estimated_Arrival_HH'] = [int(i.split(':')[0]) if str(i) != 'nan' else np.NaN for i in train['Estimated_Arrival_Time_HH:MM']]
train['Estimated_Arrival_MM'] = [int(i.split(':')[1]) if str(i) != 'nan' else np.NaN for i in train['Estimated_Arrival_Time_HH:MM']]
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,...,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Departure_HH,Estimated_Departure_MM,Estimated_Arrival_HH,Estimated_Arrival_MM
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,...,WN,19393.0,N7858A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,...,UA,20304.0,N125SY,NaN,07:40,10:24,7.0,40.0,10.0,24.0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,...,AA,19805.0,N103US,NaN,16:10,18:05,16.0,10.0,18.0,5.0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,...,UA,NaN,N595UA,NaN,09:05,17:35,9.0,5.0,17.0,35.0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,...,UA,20304.0,N161SY,NaN,09:00,10:19,9.0,0.0,10.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,...,UA,19977.0,N477UA,NaN,09:36,12:43,9.0,36.0,12.0,43.0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,...,DL,NaN,N439SW,NaN,09:20,10:28,9.0,20.0,10.0,28.0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,...,WN,19393.0,N230WN,NaN,08:00,13:40,8.0,0.0,13.0,40.0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,...,DL,19790.0,N968DL,NaN,16:13,18:24,16.0,13.0,18.0,24.0


In [9]:
test['Estimated_Departure_HH'] = [int(i.split(':')[0]) if str(i) != 'nan' else np.NaN for i in test['Estimated_Departure_Time_HH:MM']]
test['Estimated_Departure_MM'] = [int(i.split(':')[1]) if str(i) != 'nan' else np.NaN for i in test['Estimated_Departure_Time_HH:MM']]

test['Estimated_Arrival_HH'] = [int(i.split(':')[0]) if str(i) != 'nan' else np.NaN for i in test['Estimated_Arrival_Time_HH:MM']]
test['Estimated_Arrival_MM'] = [int(i.split(':')[1]) if str(i) != 'nan' else np.NaN for i in test['Estimated_Arrival_Time_HH:MM']]

In [10]:
def state(t):
	# State 채우기
	# 값이 없는 특정 State 채우기
	origin_state = train[['Origin_Airport','Origin_Airport_ID','Origin_State']].dropna()
	origin_state = origin_state.drop_duplicates()
	origin_state.columns = ['Origin_Airport','Origin_Airport_ID','F_Origin_State']
	Destination_state = train[['Destination_Airport','Destination_Airport_ID','Destination_State']].dropna()
	Destination_state = Destination_state.drop_duplicates()
	Destination_state.columns = ['Destination_Airport','Destination_Airport_ID','F_Destination_State']

	t = pd.merge(t, origin_state, how='left', on=['Origin_Airport','Origin_Airport_ID'], sort=False)
	t = pd.merge(t, Destination_state, how='left', on=['Destination_Airport','Destination_Airport_ID'], sort=False)
	

	mask = (t['Destination_Airport'] == 'YNG')&(t['Destination_Airport_ID'] == 16133)
	value = 'Indiana'
	t.loc[mask,'F_Destination_State'] = t.loc[mask,'F_Destination_State'].fillna(value) 
	t = t.drop(columns=['Origin_State', 'Destination_State'])
	t = t.rename(columns={'F_Origin_State': 'Origin_State', 'F_Destination_State': 'Destination_State'})
	return t

In [19]:
from sklearn.ensemble import RandomForestClassifier

# 준지도 학습 전 라벨링
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import OrdinalEncoder
 
train_lb = train.__deepcopy__()
test_lb = test.__deepcopy__()

train_lb = state(train_lb)
test_lb = state(test_lb)

fill_list = ['Origin_State', 'Destination_State', 'Airline', 'Carrier_ID(DOT)','Carrier_Code(IATA)','Estimated_Departure_HH','Estimated_Departure_MM', 'Estimated_Arrival_HH','Estimated_Arrival_MM','Delay']
not_labeling_list = ['Carrier_ID(DOT)','Estimated_Departure_HH','Estimated_Departure_MM', 'Estimated_Arrival_HH','Estimated_Arrival_MM']

qual_col = ['Origin_Airport', 'Destination_Airport', 'Tail_Number']
oridinary_col = ['Cancelled','Diverted','Origin_Airport','Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month','Distance']

# Train 데이터에서 null이 없는 열들 중 라벨링(not int)이 필요한 열들 순차적으로 정리
add_list = {}
for i in range(len(fill_list)): 
    if i == 0:
        add_list[i] = qual_col
    else:       
        add_list[i] = add_list[i-1] + [fill_list[i-1]]
        add_list[i] = list(set(add_list[i]) - set(not_labeling_list))

add_list

for i in range(len(fill_list)): 
    # 라벨링 준비
    label_st = train_lb.drop(columns = [fill_list[i]])
    train_st = train_lb[(train_lb[fill_list[i]].astype(str) != 'None')&(train_lb[fill_list[i]].astype(str) != 'nan')].drop(columns = [fill_list[i]])
    X_nan = train_lb[(train_lb[fill_list[i]].astype(str) == 'None')|(train_lb[fill_list[i]].astype(str) == 'nan')].drop(columns = [fill_list[i]])
    if fill_list[i] != 'Delay':
        if fill_list[i] == 'Origin_State':
            test_nan = test_lb[(test_lb[fill_list[i]].astype(str) == 'None')|(test_lb[fill_list[i]].astype(str) == 'nan')].drop(columns = [fill_list[i]])
        else:
            test_nan = test_lb[(test_lb[fill_list[i]].astype(str) == 'None')|(test_lb[fill_list[i]].astype(str) == 'nan')].drop(columns = [fill_list[i]]).dropna(subset = [fill_list[i-1]])
    else:
        pass
    print('시작 리스트: ',add_list[i])
    print('채워야하는 열: ',fill_list[i])
    
    # 라벨링이 안된 원소들 라벨링
    for j in add_list[i]:
        # train에는 없고, test에는 있는 원소는 -2 처리
        oe = OrdinalEncoder(handle_unknown='use_encoded_value',
                         unknown_value=-2)
        oe=oe.fit(label_st[j].to_numpy().reshape(-1, 1))
        train_st[j]=oe.transform(train_st[j].to_numpy().reshape(-1, 1))
        if fill_list[i] == 'Origin_State':
            pass
        else:
            if len(X_nan[fill_list[i-1]]) != 0:
                X_nan[j]=oe.transform(X_nan[j].to_numpy().reshape(-1, 1))
            else:
                pass
        if fill_list[i] != 'Delay':
            test_nan[j]= oe.transform(test_nan[j].to_numpy().reshape(-1, 1))
        else:
            pass

    # 라벨링을 통해 null이 없었던 columns을 통해서 self_training
    X = train_st[oridinary_col]
    target_col = oridinary_col+ [fill_list[i]]
    y = train_lb[(train_lb[fill_list[i]].astype(str) != 'None')&(train_lb[fill_list[i]].astype(str) != 'nan')][target_col][fill_list[i]]

    stclf = SelfTrainingClassifier(
        base_estimator = RandomForestClassifier(n_estimators = 100, random_state=113),
        verbose = True)
    stclf.fit(X, y)   

    # 결과값 삽입
    if fill_list[i] == 'Origin_State':
        pass
    else:
        if len(X_nan[fill_list[i-1]]) != 0:
            X_nan[fill_list[i]+'_pred'] = stclf.predict(X_nan[oridinary_col])
        else:
            pass
        
    if fill_list[i] != 'Delay':
        test_nan[fill_list[i]+'_pred'] = stclf.predict(test_nan[oridinary_col])
    else:    
        pass
    if fill_list[i] == 'Origin_State':
        pass
    else:
        if len(X_nan[fill_list[i-1]]) != 0:
            train_lb.loc[(train_lb[fill_list[i]].astype(str) == 'None')|(train_lb[fill_list[i]].astype(str) == 'nan'), fill_list[i]] = X_nan[fill_list[i]+'_pred']
        else:    
            pass
    
    if fill_list[i] != 'Delay':
        test_lb.loc[(test_lb[fill_list[i]].astype(str) == 'None')|(test_lb[fill_list[i]].astype(str) ==  'nan'), fill_list[i]] = test_nan[fill_list[i]+'_pred']
    else:    
        pass
    
    # 다음 self_training을 위해 열 변경
    oridinary_col = oridinary_col + [fill_list[i]]
    print('다음 훈련 리스트: ', oridinary_col)


시작 리스트:  ['Origin_Airport', 'Destination_Airport', 'Tail_Number']
채워야하는 열:  Origin_State


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


다음 훈련 리스트:  ['Cancelled', 'Diverted', 'Origin_Airport', 'Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month', 'Distance', 'Origin_State']
시작 리스트:  ['Origin_State', 'Destination_Airport', 'Tail_Number', 'Origin_Airport']
채워야하는 열:  Destination_State


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


다음 훈련 리스트:  ['Cancelled', 'Diverted', 'Origin_Airport', 'Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month', 'Distance', 'Origin_State', 'Destination_State']
시작 리스트:  ['Destination_State', 'Tail_Number', 'Origin_State', 'Destination_Airport', 'Origin_Airport']
채워야하는 열:  Airline


...

다음 훈련 리스트:  ['Cancelled', 'Diverted', 'Origin_Airport', 'Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month', 'Distance', 'Origin_State', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Carrier_Code(IATA)', 'Estimated_Departure_HH', 'Estimated_Departure_MM', 'Estimated_Arrival_HH']
시작 리스트:  ['Airline', 'Destination_State', 'Carrier_Code(IATA)', 'Tail_Number', 'Origin_State', 'Destination_Airport', 'Origin_Airport']
채워야하는 열:  Estimated_Arrival_MM


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


다음 훈련 리스트:  ['Cancelled', 'Diverted', 'Origin_Airport', 'Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month', 'Distance', 'Origin_State', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Carrier_Code(IATA)', 'Estimated_Departure_HH', 'Estimated_Departure_MM', 'Estimated_Arrival_HH', 'Estimated_Arrival_MM']
시작 리스트:  ['Airline', 'Destination_State', 'Carrier_Code(IATA)', 'Tail_Number', 'Origin_State', 'Destination_Airport', 'Origin_Airport']
채워야하는 열:  Delay


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


다음 훈련 리스트:  ['Cancelled', 'Diverted', 'Origin_Airport', 'Origin_Airport_ID', 'Destination_Airport', 'Destination_Airport_ID', 'Tail_Number', 'Month', 'Day_of_Month', 'Distance', 'Origin_State', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Carrier_Code(IATA)', 'Estimated_Departure_HH', 'Estimated_Departure_MM', 'Estimated_Arrival_HH', 'Estimated_Arrival_MM', 'Delay']


In [20]:
train_lb.to_csv(os.path.join(file_dir,'train_lb.csv'), index=False,encoding="utf-8-sig")
test_lb.to_csv(os.path.join(file_dir,'test_lb.csv'), index=False,encoding="utf-8-sig")

In [21]:
train = train_lb
test = test_lb

In [22]:
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Destination_Airport,...,Tail_Number,Delay,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Estimated_Departure_HH,Estimated_Departure_MM,Estimated_Arrival_HH,Estimated_Arrival_MM,Origin_State,Destination_State
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,HOU,...,N7858A,Not_Delayed,NaN,NaN,14.0,55.0,16.0,20.0,Oklahoma,Texas
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,SLC,...,N125SY,Not_Delayed,07:40,10:24,7.0,40.0,10.0,24.0,Illinois,Utah
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,LGA,...,N103US,Not_Delayed,16:10,18:05,16.0,10.0,18.0,5.0,North Carolina,New York
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,EWR,...,N595UA,Not_Delayed,09:05,17:35,9.0,5.0,17.0,35.0,California,New Jersey
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,ACV,...,N161SY,Not_Delayed,09:00,10:19,9.0,0.0,10.0,19.0,California,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,PHL,...,N477UA,Not_Delayed,09:36,12:43,9.0,36.0,12.0,43.0,Illinois,Pennsylvania
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,MSP,...,N439SW,Not_Delayed,09:20,10:28,9.0,20.0,10.0,28.0,North Dakota,Minnesota
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,HOU,...,N230WN,Not_Delayed,08:00,13:40,8.0,0.0,13.0,40.0,California,Texas
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,ATL,...,N968DL,Not_Delayed,16:13,18:24,16.0,13.0,18.0,24.0,Tennessee,Georgia


In [23]:
train = train.drop(columns = ['Estimated_Departure_Time','Estimated_Arrival_Time','Estimated_Departure_Time_HH:MM', 'Estimated_Arrival_Time_HH:MM'])
train['Estimated_Departure_Time'] = [int(str(int(i)) + str(int(j))) for i, j in zip(train['Estimated_Departure_HH'], train['Estimated_Departure_MM'])]
train['Estimated_Arrival_Time'] = [int(str(int(i)) + str(int(j))) for i, j in zip(train['Estimated_Arrival_HH'], train['Estimated_Arrival_MM'])]
train

,ID,Month,Day_of_Month,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Destination_Airport,Destination_Airport_ID,Distance,...,Tail_Number,Delay,Estimated_Departure_HH,Estimated_Departure_MM,Estimated_Arrival_HH,Estimated_Arrival_MM,Origin_State,Destination_State,Estimated_Departure_Time,Estimated_Arrival_Time
0,TRAIN_000000,4,15,0,0,OKC,13851,HOU,12191,419.0,...,N7858A,Not_Delayed,14.0,55.0,16.0,20.0,Oklahoma,Texas,1455,1620
1,TRAIN_000001,8,15,0,0,ORD,13930,SLC,14869,1250.0,...,N125SY,Not_Delayed,7.0,40.0,10.0,24.0,Illinois,Utah,740,1024
2,TRAIN_000002,9,6,0,0,CLT,11057,LGA,12953,544.0,...,N103US,Not_Delayed,16.0,10.0,18.0,5.0,North Carolina,New York,1610,185
3,TRAIN_000003,7,10,0,0,LAX,12892,EWR,11618,2454.0,...,N595UA,Not_Delayed,9.0,5.0,17.0,35.0,California,New Jersey,95,1735
4,TRAIN_000004,1,11,0,0,SFO,14771,ACV,10157,250.0,...,N161SY,Not_Delayed,9.0,0.0,10.0,19.0,California,California,90,1019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,0,0,ORD,13930,PHL,14100,678.0,...,N477UA,Not_Delayed,9.0,36.0,12.0,43.0,Illinois,Pennsylvania,936,1243
999996,TRAIN_999996,5,30,0,0,FAR,11637,MSP,13487,223.0,...,N439SW,Not_Delayed,9.0,20.0,10.0,28.0,North Dakota,Minnesota,920,1028
999997,TRAIN_999997,6,28,0,0,OAK,13796,HOU,12191,1642.0,...,N230WN,Not_Delayed,8.0,0.0,13.0,40.0,California,Texas,80,1340
999998,TRAIN_999998,9,27,0,0,BNA,10693,ATL,10397,214.0,...,N968DL,Not_Delayed,16.0,13.0,18.0,24.0,Tennessee,Georgia,1613,1824


In [24]:
test = test.drop(columns = ['Estimated_Departure_Time','Estimated_Arrival_Time','Estimated_Departure_Time_HH:MM', 'Estimated_Arrival_Time_HH:MM'])
test['Estimated_Departure_Time'] = [int(str(int(i)) + str(int(j))) for i, j in zip(test['Estimated_Departure_HH'], test['Estimated_Departure_MM'])]
test['Estimated_Arrival_Time'] = [int(str(int(i)) + str(int(j))) for i, j in zip(test['Estimated_Arrival_HH'], test['Estimated_Arrival_MM'])]
test

,ID,Month,Day_of_Month,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Destination_Airport,Destination_Airport_ID,Distance,...,Carrier_ID(DOT),Tail_Number,Estimated_Departure_HH,Estimated_Departure_MM,Estimated_Arrival_HH,Estimated_Arrival_MM,Origin_State,Destination_State,Estimated_Departure_Time,Estimated_Arrival_Time
0,TEST_000000,12,16,0,0,IAH,12266,SAT,14683,191.0,...,19977.0,N79402,11.0,56.0,12.0,54.0,Texas,Texas,1156,1254
1,TEST_000001,9,12,0,0,EWR,11618,ATL,10397,746.0,...,19790.0,N3765,15.0,0.0,17.0,15.0,New Jersey,Georgia,150,1715
2,TEST_000002,3,6,0,0,ORD,13930,LGA,12953,733.0,...,19977.0,N413UA,16.0,0.0,19.0,15.0,Illinois,New York,160,1915
3,TEST_000003,5,18,0,0,OAK,13796,LAX,12892,337.0,...,19393.0,N905WN,19.0,20.0,20.0,45.0,California,California,1920,2045
4,TEST_000004,7,7,0,0,FLL,11697,LAX,12892,2343.0,...,20409.0,N945JT,19.0,15.0,21.0,52.0,Florida,California,1915,2152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TEST_999995,6,2,0,0,LAX,12892,DEN,11292,862.0,...,19393.0,N720WN,17.0,45.0,22.0,40.0,California,Colorado,1745,2240
999996,TEST_999996,6,8,0,0,BUF,10792,ORD,13930,473.0,...,19977.0,N401UA,6.0,0.0,6.0,48.0,New York,Illinois,60,648
999997,TEST_999997,6,11,0,0,IAD,12264,DTW,11433,383.0,...,20378.0,N510MJ,17.0,10.0,19.0,7.0,Virginia,Michigan,1710,197
999998,TEST_999998,11,17,0,0,SAN,14679,BOS,10721,2588.0,...,20409.0,N986JB,21.0,10.0,5.0,0.0,California,Massachusetts,2110,50


In [26]:
import itertools
import pandas as pd
import datetime 
def feature_engineering(t):
	def to_time(time_list):
		Time = pd.Series(time_list).astype(str).str.zfill(4)
		Time = Time.replace('2400','0000')
		return Time
	time_list = [i if str(i) == 'nan' else str(int(i))  for i in t['Estimated_Departure_Time'] ]
	time_list1 = [i if str(i) == 'nan' else str(int(i))  for i in t['Estimated_Arrival_Time'] ]
	
	def parse_times(val):
		val = val.strip()
		h, m = int(val[:2]), int(val[2:])
		if hrs:= m // 60>0:
			h += hrs
			m = m - hrs*60
		h = h%24
		return datetime.time(hour=h, minute=m).strftime('%H:%M')

	t['Estimated_Departure_Time_HH:MM'] = [parse_times(i) for i in to_time(time_list)] 
	t['Estimated_Arrival_Time_HH:MM'] = [parse_times(i) for i in to_time(time_list1)] 
	
	# 예상 비행시간 만들기 (분으로 만들기)
	est_time = []
	for i,j in zip(t['Estimated_Arrival_Time_HH:MM'], t['Estimated_Departure_Time_HH:MM']):
		if i != 'nan' and j != 'nan':
			if (datetime.datetime.strptime(i, "%H:%M") - datetime.datetime.strptime(j, "%H:%M")).total_seconds()/60 >= 0:
				est_time.append((datetime.datetime.strptime(i, "%H:%M") - datetime.datetime.strptime(j, "%H:%M")).total_seconds()/60)
			else:
				time = datetime.datetime.strptime(i, "%H:%M") - datetime.datetime.strptime(j, "%H:%M") + datetime.datetime.strptime('23:59', "%H:%M") + datetime.timedelta(minutes=1)
				est_time.append(datetime.timedelta(hours=time.hour,minutes=time.minute ).total_seconds()/60)
		else:
			est_time.append(np.NaN)
			
	# 예상 출발, 도착시간 그룹 화
	def time_gb(x):		
		if x >= 600 and x <= 659:
			return '0600-0659'
		elif x>=1400 and x<=1459:
			return '1400-1459'
		elif x>=1200 and x<=1259:
			return '1200-1259'
		elif x>=1500 and x<=1559:
			return '1500-1559'
		elif x>=1900 and x<=1959:
			return '1900-1959'
		elif x>=900 and x<=959:
			return '0900-0959'
		elif x>=1000 and x<=1059:
			return  '1000-1059'
		elif x>=2000 and x<=2059:
			return '2000-2059'
		elif x>=1300 and x<=1359:
			return '1300-1359'
		elif x>=1100 and x<=1159:
			return '1100-1159'
		elif x>=800 and x<=859:
			return '0800-0859'
		elif x>=2200 and x<=2259:
			return '2200-2259'
		elif x>=1600 and x<=1659:
			return '1600-1659'
		elif x>=1700 and x<=1759:
			return '1700-1759'
		elif x>=2100 and x<=2159:
			return '2100-2159'
		elif x>=700 and x<=759:
			return '0700-0759'
		elif x>=1800 and x<=1859:
			return '1800-1859'
		elif x>=1 and x<=559:
			return '0001-0559'
		elif x>=2300 and x<=2400:
			return '2300-2400'
		else:
			return 'dummy'

	t['Dep_time_gb'] = t['Estimated_Departure_Time'].dropna().apply(time_gb)
	t['Arr_time_gb'] = t['Estimated_Arrival_Time'].dropna().apply(time_gb)
	
	# 예상 출발, 도착시간 그룹 화
	def distance_gb(x):
		if x < 700 :
			return 'short_route'
		elif x>=700 and x<3000:
			return 'mid_route'
		elif x>=3000:
			return 'long_route'  
		else:
			return 'dummy'
	t['route_gb'] = t['Distance'].apply(distance_gb)
	
	return t

In [27]:
train = feature_engineering(train)
test = feature_engineering(test)

In [28]:
train

,ID,Month,Day_of_Month,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Destination_Airport,Destination_Airport_ID,Distance,...,Estimated_Arrival_MM,Origin_State,Destination_State,Estimated_Departure_Time,Estimated_Arrival_Time,Estimated_Departure_Time_HH:MM,Estimated_Arrival_Time_HH:MM,Dep_time_gb,Arr_time_gb,route_gb
0,TRAIN_000000,4,15,0,0,OKC,13851,HOU,12191,419.0,...,20.0,Oklahoma,Texas,1455,1620,14:55,16:20,1400-1459,1600-1659,short_route
1,TRAIN_000001,8,15,0,0,ORD,13930,SLC,14869,1250.0,...,24.0,Illinois,Utah,740,1024,07:40,10:24,0700-0759,1000-1059,mid_route
2,TRAIN_000002,9,6,0,0,CLT,11057,LGA,12953,544.0,...,5.0,North Carolina,New York,1610,185,16:10,02:25,1600-1659,0001-0559,short_route
3,TRAIN_000003,7,10,0,0,LAX,12892,EWR,11618,2454.0,...,35.0,California,New Jersey,95,1735,01:35,17:35,0001-0559,1700-1759,mid_route
4,TRAIN_000004,1,11,0,0,SFO,14771,ACV,10157,250.0,...,19.0,California,California,90,1019,01:30,10:19,0001-0559,1000-1059,short_route
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,0,0,ORD,13930,PHL,14100,678.0,...,43.0,Illinois,Pennsylvania,936,1243,09:36,12:43,0900-0959,1200-1259,short_route
999996,TRAIN_999996,5,30,0,0,FAR,11637,MSP,13487,223.0,...,28.0,North Dakota,Minnesota,920,1028,09:20,10:28,0900-0959,1000-1059,short_route
999997,TRAIN_999997,6,28,0,0,OAK,13796,HOU,12191,1642.0,...,40.0,California,Texas,80,1340,01:20,13:40,0001-0559,1300-1359,mid_route
999998,TRAIN_999998,9,27,0,0,BNA,10693,ATL,10397,214.0,...,24.0,Tennessee,Georgia,1613,1824,16:13,18:24,1600-1659,1800-1859,short_route


In [29]:
map = {
    'Not_Delayed' :    1,
    'Delayed'     :    0, 
}
train['Delay'] = train.Delay.map(map)


In [30]:
from sklearn.preprocessing import OrdinalEncoder
# train 라벨링
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Carrier_Code(IATA)', 'Tail_Number', 'Estimated_Departure_Time_HH:MM','Estimated_Arrival_Time_HH:MM', 'Dep_time_gb',	'Arr_time_gb',	'route_gb']


for i in qual_col:
    # train에는 없고, test에는 있는 원소는 -2 처리
    oe = OrdinalEncoder(handle_unknown='use_encoded_value',
                         unknown_value=-2)
    
    oe=oe.fit(train[i].to_numpy().reshape(-1, 1))
    train[i] = oe.transform(train[i].to_numpy().reshape(-1, 1))
    test[i] = oe.transform(test[i].to_numpy().reshape(-1, 1))

In [31]:
X_train = train.drop(columns = ['ID','Delay'], axis=1)
y_train = train['Delay']
x_test = test.drop(columns = ['ID'], axis=1)

In [32]:
params = {'bootstrap_type' : 'Poisson',
          'early_stopping_rounds' : 100,
          'iterations' : 3000,
          'custom_metric':['Logloss','AUC'],
          'use_best_model': True,
          'task_type':"GPU",
          'bagging_temperature':1,
          'random_state' : 113,
          'verbose':100}

In [33]:
from catboost import CatBoostClassifier 

model = CatBoostClassifier(**params)

In [34]:
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import Pool

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=113)
y_valid_pred = 0*y_train
y_preds1 = np.zeros(x_test.shape[0])
y_preds2 = np.zeros(x_test.shape[0])

qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_ID(DOT)', 'Carrier_Code(IATA)', 'Tail_Number', 'Estimated_Departure_Time_HH:MM','Estimated_Arrival_Time_HH:MM', 'Dep_time_gb',	'Arr_time_gb',	'route_gb']

for i, (train_index, test_index) in enumerate(skfold.split(X_train, y_train)):    
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    train_data = Pool(data=X_train_fold.astype('int'), label=y_train_fold, cat_features=qual_col)
    valid_data = Pool(data=X_valid_fold.astype('int'), label=y_valid_fold, cat_features=qual_col)

    print( "\nFold ", i)
    
    fit_model = model.fit(train_data, 
                          eval_set=valid_data,
                          use_best_model=True
                          )
    print( "  N trees = ", model.tree_count_ )
    
    # Predict value Clipping
    y_preds1 +=  fit_model.predict_proba(x_test[X_valid_fold.columns].astype('int'))[:,0]
    y_preds2 +=  fit_model.predict_proba(x_test[X_valid_fold.columns].astype('int'))[:,1]
    
y_preds1 /= 5  # Average test set predictions
y_preds2 /= 5


Fold  0
Learning rate set to 0.0256


Default metric period is 5 because AUC is/are not implemented for GPU


0:	learn: 0.6654264	test: 0.6654283	best: 0.6654283 (0)	total: 92.8ms	remaining: 4m 38s
100:	learn: 0.2568778	test: 0.2566841	best: 0.2566841 (100)	total: 9.95s	remaining: 4m 45s
200:	learn: 0.2550330	test: 0.2547385	best: 0.2547385 (200)	total: 21s	remaining: 4m 52s
300:	learn: 0.2546718	test: 0.2543343	best: 0.2543343 (300)	total: 32.3s	remaining: 4m 49s
400:	learn: 0.2543668	test: 0.2539669	best: 0.2539669 (400)	total: 44.5s	remaining: 4m 48s
500:	learn: 0.2540136	test: 0.2535373	best: 0.2535373 (500)	total: 58.3s	remaining: 4m 50s
600:	learn: 0.2537687	test: 0.2532646	best: 0.2532646 (600)	total: 1m 11s	remaining: 4m 46s
700:	learn: 0.2535684	test: 0.2530122	best: 0.2530121 (699)	total: 1m 24s	remaining: 4m 37s
800:	learn: 0.2533925	test: 0.2527887	best: 0.2527887 (800)	total: 1m 37s	remaining: 4m 28s
900:	learn: 0.2532573	test: 0.2526553	best: 0.2526553 (900)	total: 1m 51s	remaining: 4m 19s
1000:	learn: 0.2529206	test: 0.2522449	best: 0.2522448 (999)	total: 2m 4s	remaining: 4m 9s


Default metric period is 5 because AUC is/are not implemented for GPU


0:	learn: 0.6654270	test: 0.6654240	best: 0.6654240 (0)	total: 138ms	remaining: 6m 53s
100:	learn: 0.2570003	test: 0.2568070	best: 0.2568070 (100)	total: 13.3s	remaining: 6m 20s
200:	learn: 0.2552166	test: 0.2549290	best: 0.2549290 (200)	total: 26.5s	remaining: 6m 9s
300:	learn: 0.2549342	test: 0.2545928	best: 0.2545928 (300)	total: 39.8s	remaining: 5m 57s
400:	learn: 0.2546782	test: 0.2542514	best: 0.2542514 (400)	total: 53.1s	remaining: 5m 44s
500:	learn: 0.2544335	test: 0.2539128	best: 0.2539128 (500)	total: 1m 6s	remaining: 5m 33s
600:	learn: 0.2542494	test: 0.2537035	best: 0.2537035 (600)	total: 1m 20s	remaining: 5m 20s
700:	learn: 0.2541024	test: 0.2535053	best: 0.2535053 (700)	total: 1m 33s	remaining: 5m 5s
800:	learn: 0.2539427	test: 0.2533071	best: 0.2533071 (800)	total: 1m 46s	remaining: 4m 52s
900:	learn: 0.2538337	test: 0.2531923	best: 0.2531923 (900)	total: 2m	remaining: 4m 39s
1000:	learn: 0.2536385	test: 0.2529188	best: 0.2529188 (1000)	total: 2m 13s	remaining: 4m 26s
11

...

0:	learn: 0.6654257	test: 0.6654273	best: 0.6654273 (0)	total: 145ms	remaining: 7m 15s
100:	learn: 0.2569417	test: 0.2567949	best: 0.2567949 (100)	total: 13.6s	remaining: 6m 30s
200:	learn: 0.2551175	test: 0.2549241	best: 0.2549241 (200)	total: 26.9s	remaining: 6m 14s
300:	learn: 0.2548063	test: 0.2545946	best: 0.2545946 (300)	total: 40.2s	remaining: 6m
400:	learn: 0.2545177	test: 0.2542641	best: 0.2542641 (400)	total: 53.4s	remaining: 5m 46s
500:	learn: 0.2542894	test: 0.2539597	best: 0.2539597 (500)	total: 1m 6s	remaining: 5m 33s
600:	learn: 0.2541221	test: 0.2537475	best: 0.2537475 (600)	total: 1m 20s	remaining: 5m 19s
700:	learn: 0.2539125	test: 0.2535013	best: 0.2535013 (700)	total: 1m 33s	remaining: 5m 5s
800:	learn: 0.2537633	test: 0.2533103	best: 0.2533103 (798)	total: 1m 46s	remaining: 4m 52s
900:	learn: 0.2536522	test: 0.2531921	best: 0.2531921 (900)	total: 1m 59s	remaining: 4m 39s
1000:	learn: 0.2533863	test: 0.2528730	best: 0.2528729 (999)	total: 2m 13s	remaining: 4m 26s
11

Default metric period is 5 because AUC is/are not implemented for GPU


0:	learn: 0.6654308	test: 0.6654295	best: 0.6654295 (0)	total: 144ms	remaining: 7m 12s
100:	learn: 0.2569821	test: 0.2567575	best: 0.2567575 (100)	total: 13.5s	remaining: 6m 28s
200:	learn: 0.2551845	test: 0.2548147	best: 0.2548147 (200)	total: 26.8s	remaining: 6m 12s
300:	learn: 0.2549151	test: 0.2544784	best: 0.2544784 (300)	total: 40s	remaining: 5m 58s
400:	learn: 0.2546554	test: 0.2541284	best: 0.2541284 (400)	total: 53.2s	remaining: 5m 44s
500:	learn: 0.2544175	test: 0.2537304	best: 0.2537304 (500)	total: 1m 6s	remaining: 5m 33s
600:	learn: 0.2542564	test: 0.2535095	best: 0.2535095 (599)	total: 1m 20s	remaining: 5m 19s
700:	learn: 0.2540740	test: 0.2532362	best: 0.2532362 (699)	total: 1m 33s	remaining: 5m 5s
800:	learn: 0.2539454	test: 0.2530407	best: 0.2530406 (799)	total: 1m 46s	remaining: 4m 52s
900:	learn: 0.2538469	test: 0.2529200	best: 0.2529200 (900)	total: 1m 59s	remaining: 4m 39s
1000:	learn: 0.2536360	test: 0.2526024	best: 0.2526024 (1000)	total: 2m 13s	remaining: 4m 25s

Default metric period is 5 because AUC is/are not implemented for GPU


0:	learn: 0.6654257	test: 0.6654257	best: 0.6654257 (0)	total: 142ms	remaining: 7m 4s
100:	learn: 0.2569376	test: 0.2567638	best: 0.2567638 (100)	total: 14.3s	remaining: 6m 49s
200:	learn: 0.2551332	test: 0.2548437	best: 0.2548437 (200)	total: 29s	remaining: 6m 44s
300:	learn: 0.2548421	test: 0.2544831	best: 0.2544831 (300)	total: 43.7s	remaining: 6m 31s
400:	learn: 0.2545554	test: 0.2540878	best: 0.2540878 (400)	total: 57.9s	remaining: 6m 15s
500:	learn: 0.2543032	test: 0.2537124	best: 0.2537124 (500)	total: 1m 12s	remaining: 5m 59s
600:	learn: 0.2541472	test: 0.2535379	best: 0.2535379 (600)	total: 1m 26s	remaining: 5m 45s
700:	learn: 0.2539617	test: 0.2532901	best: 0.2532901 (700)	total: 1m 40s	remaining: 5m 29s
800:	learn: 0.2538219	test: 0.2530999	best: 0.2530999 (800)	total: 1m 54s	remaining: 5m 15s
900:	learn: 0.2537253	test: 0.2529896	best: 0.2529896 (900)	total: 2m 9s	remaining: 5m 2s
1000:	learn: 0.2535051	test: 0.2526777	best: 0.2526776 (999)	total: 2m 25s	remaining: 4m 50s
1

In [35]:
x_test['Not_Delayed'] =  y_preds1
x_test['Delayed'] =  y_preds2
x_test = x_test.reset_index()

In [39]:
sample_submission = sample_submission.reset_index()
sample_submission = sample_submission.drop(columns = ['Not_Delayed','Delayed'])
sample_submission

KeyError: "['Not_Delayed', 'Delayed'] not found in axis"

In [40]:
sample_submission

,index,ID
0,0,TEST_000000
1,1,TEST_000001
2,2,TEST_000002
3,3,TEST_000003
4,4,TEST_000004
...,...,...
999995,999995,TEST_999995
999996,999996,TEST_999996
999997,999997,TEST_999997
999998,999998,TEST_999998


In [41]:
sample_submission = pd.merge(sample_submission, x_test[['index', 'Not_Delayed','Delayed']], left_on = 'index', right_on = 'index', how = 'inner')

In [42]:
sample_submission = sample_submission[['ID', 'Not_Delayed','Delayed']] 
sample_submission.set_index('ID')
sample_submission

,ID,Not_Delayed,Delayed
0,TEST_000000,0.068779,0.931221
1,TEST_000001,0.066999,0.933001
2,TEST_000002,0.062901,0.937099
3,TEST_000003,0.065889,0.934111
4,TEST_000004,0.081581,0.918419
...,...,...,...
999995,TEST_999995,0.077089,0.922911
999996,TEST_999996,0.043862,0.956138
999997,TEST_999997,0.065478,0.934522
999998,TEST_999998,0.113001,0.886999


In [43]:
sample_submission.to_csv(os.path.join(file_dir,'submit_cat22.csv'), index=False,encoding="utf-8-sig")